In [238]:
import warnings

In [239]:
warnings.filterwarnings('ignore')

In [240]:
import pandas as pd 

In [241]:
data=pd.read_csv(r'C:\Users\deyar\OneDrive\Desktop\InsurencepPricePrediction\notebooks\data\insurance.csv')

In [242]:
data.head()

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46
3,33,male,22.7,0,no,northwest,21984.47
4,32,male,28.9,0,no,northwest,3866.86


In [243]:
data.shape

(1338, 7)

In [244]:
data.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
expenses    0
dtype: int64

In [245]:
data.duplicated().sum()

1

In [246]:
data=data.drop_duplicates()

In [247]:
data.duplicated().sum()

0

In [248]:
cat=[]
num=[]
for i in data.columns:
    if data[i].dtype=="object":
        cat.append(i)
    else:
        num.append(i)

In [249]:
cat

['sex', 'smoker', 'region']

In [250]:
sex=['female', 'male']

In [251]:
region=['southwest', 'southeast', 'northwest', 'northeast']

In [252]:
smoker=['no','yes']

In [253]:
num

['age', 'bmi', 'children', 'expenses']

In [254]:
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,StandardScaler,MinMaxScaler

In [255]:
from sklearn.compose import ColumnTransformer

In [256]:
from sklearn.pipeline import Pipeline

In [257]:
from sklearn.impute import SimpleImputer

In [258]:
data.head()

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46
3,33,male,22.7,0,no,northwest,21984.47
4,32,male,28.9,0,no,northwest,3866.86


In [259]:
cat

['sex', 'smoker', 'region']

In [260]:
num

['age', 'bmi', 'children', 'expenses']

In [261]:
num.remove('expenses')

In [262]:
num

['age', 'bmi', 'children']

In [263]:
num_pipeline=Pipeline(
                steps=[
                ('imputer',SimpleImputer(strategy='median')),
                ('scale',MinMaxScaler())])
cat_pipeline=Pipeline(
                steps=[
                ('imputer',SimpleImputer(strategy='most_frequent')),
                ('ordinalencoder',OrdinalEncoder(categories=[sex,smoker,region])),
                ('scale',MinMaxScaler())])
preprocessor=ColumnTransformer([
            ('num_pipeline',num_pipeline,num),
            ('cat_pipeline',cat_pipeline,cat)])

In [264]:
x=data.drop('expenses',axis=1)

In [265]:
y=data['expenses']

In [266]:
m=preprocessor.fit_transform(x)

In [267]:
f=pd.DataFrame(m,columns=x.columns)

In [268]:
f.head()

,age,sex,bmi,children,smoker,region
0,0.021739,0.320755,0.0,0.0,1.0,0.000000
1,0.000000,0.479784,0.2,1.0,0.0,0.333333
2,0.217391,0.458221,0.6,1.0,0.0,0.333333
3,0.326087,0.180593,0.0,1.0,0.0,0.666667
4,0.304348,0.347709,0.0,1.0,0.0,0.666667


In [269]:
f.describe()

,age,sex,bmi,children,smoker,region
count,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000
mean,0.461351,0.395297,0.219147,0.504862,0.204936,0.494640
std,0.305312,0.164438,0.241114,0.500163,0.403806,0.368403
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.195652,0.277628,0.000000,0.000000,0.000000,0.333333
50%,0.456522,0.388140,0.200000,1.000000,0.000000,0.333333
75%,0.717391,0.504043,0.400000,1.000000,0.000000,0.666667
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [270]:
data1=pd.concat([f,y],axis=1)

In [271]:
data1.head()

,age,sex,bmi,children,smoker,region,expenses
0,0.021739,0.320755,0.0,0.0,1.0,0.000000,16884.92
1,0.000000,0.479784,0.2,1.0,0.0,0.333333,1725.55
2,0.217391,0.458221,0.6,1.0,0.0,0.333333,4449.46
3,0.326087,0.180593,0.0,1.0,0.0,0.666667,21984.47
4,0.304348,0.347709,0.0,1.0,0.0,0.666667,3866.86


In [272]:
from sklearn.model_selection import train_test_split

In [273]:
x_trian,x_test,y_train,y_test=train_test_split(f,y,random_state=11,test_size=.2)

In [274]:
from sklearn.ensemble import AdaBoostRegressor

In [275]:
a=AdaBoostRegressor()

In [276]:
a.fit(x_trian,y_train)

AdaBoostRegressor()

In [277]:
p=a.predict(x_test)

In [278]:
from sklearn.metrics import r2_score,mean_absolute_error

In [279]:
r2_score(y_test,p)

0.8489675277959782

In [280]:
import optuna

In [284]:
def objective(trial):
    learning_rate=trial.suggest_float('learning_rate',.001,1,log=True)
    loss=trial.suggest_categorical('loss',['linear','square','exponential'])
    n_estimators=trial.suggest_int('n_estimators',100,1000)
    model=AdaBoostRegressor(n_estimators=n_estimators,
    learning_rate=learning_rate,
    loss=loss
    )
    model.fit(x_trian,y_train)
    p=model.predict(x_test)
    v=r2_score(y_test,p)
    return v

In [285]:
study=optuna.create_study(direction='maximize')

[I 2024-04-19 05:00:23,686] A new study created in memory with name: no-name-8dab0243-fb9c-4b00-a71d-aa77704cfdd6


In [286]:
study.optimize(objective,n_trials=100)

[I 2024-04-19 05:00:24,713] Trial 0 finished with value: 0.8738118723034412 and parameters: {'learning_rate': 0.9431691095417283, 'loss': 'linear', 'n_estimators': 983}. Best is trial 0 with value: 0.8738118723034412.
[I 2024-04-19 05:00:24,977] Trial 1 finished with value: 0.6554551306875172 and parameters: {'learning_rate': 0.1614575651023029, 'loss': 'exponential', 'n_estimators': 168}. Best is trial 0 with value: 0.8738118723034412.
[I 2024-04-19 05:00:25,315] Trial 2 finished with value: 0.8544776268090437 and parameters: {'learning_rate': 0.026312948293508144, 'loss': 'linear', 'n_estimators': 528}. Best is trial 0 with value: 0.8738118723034412.
[I 2024-04-19 05:00:26,429] Trial 3 finished with value: 0.8333557875984653 and parameters: {'learning_rate': 0.004975364137272745, 'loss': 'square', 'n_estimators': 780}. Best is trial 0 with value: 0.8738118723034412.
[I 2024-04-19 05:00:27,096] Trial 4 finished with value: 0.8553418890035747 and parameters: {'learning_rate': 0.0121367

In [287]:
study.best_params

{'learning_rate': 0.0017318756129784008,
 'loss': 'exponential',
 'n_estimators': 116}

In [289]:
study.best_value

0.9148659538632945